# Assignment 2 - Data from the web

## Data scraping

## 1. Obtain all the data

In [154]:
import requests
import pandas as pd
import numpy as np
import os
import os.path
import re
from IPython.display import display
from bs4 import BeautifulSoup

The first thing to do is to get the base URL (basically looking at the address bar of the browser).
We need a second variable INDEX_PARAM because this page is using frames, and we'll need the base address to access them.

In [155]:
BASE_URL = "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm"
INDEX_PARAM = "ww_i_reportmodel=133685247"

Then let's fetch the parameters names, but as it is a bit tricker, BeautifulSoup will be a great help for this (and also some manual html inspection).

In [156]:
r = requests.get(BASE_URL + '?' + INDEX_PARAM)
soup = BeautifulSoup(r.text, 'html.parser')


#print(soup)

for frame in soup.find_all('frame'):
    if frame['name'] == 'toc':
        r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/" + frame['src'])
        print(frame['src'])
# The content of the frame

# Print slects
soup = BeautifulSoup(r.text, 'html.parser')
query = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247'
    


!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247


For convenience, we also create another function that builds the query from some desired parameters.

In [157]:
def build_query(section, startYear, endYear, semesterType, semesterNumber, semesterSeason):
    # Print slects
    soup = BeautifulSoup(r.text, 'html.parser')
    query = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247'


    for in_put in soup.findAll('input'):
        #print (in_put['name'])
        if in_put.text == "xls":
        #print(in_put['value'])
            query = query + '&' + in_put['name'] + '=' + in_put['value']
     
    for select in soup.findAll('select'):
    #print (select['name'])
        for t in select.findAll('option'):
        #print(t)
            if t.text == section:
            #print(t['value'])
                query = query + '&' + select['name'] + '=' + t['value']
            if t.text == str(startYear) + '-' + str(endYear):
                query = query + '&' + select['name'] + '=' + t['value']
            if t.text == semesterType + " " + str(semesterNumber):
                query = query + '&' + select['name'] + '=' + t['value']
            if t.text == "Semestre " + semesterSeason:
                query = query + '&' + select['name'] + '=' + t['value']
    return query


And finally a nice function to easily query the EPFL page and get the data from it.

In [190]:
DATA_FOLDER_PATH = 'Data/'
def query_epfl(section, startYear, endYear, semesterType, semesterNumber, semesterSeason, update = False, writeToFile = True):
    #we create files to save the queries (and network bandwidth)
    path = str(section) + str(startYear) + str(endYear) + str(semesterType) + str(semesterNumber) + str(semesterSeason)
    
    #to avoid file naming issues
    path = re.sub('[^A-Za-z0-9]+', '', path)
    path += ".xls"
    
    df = None
    #we query the epfl server only if the file does not exist, or if we want to update the data
    if(not os.path.isfile(DATA_FOLDER_PATH + path) or update):
        queryString = build_query(section, startYear, endYear, semesterType, semesterNumber, semesterSeason)
        response = requests.get(queryString, stream=True)
        if response.status_code == 200:
            data = pd.read_html(response.text, match='.+', flavor=None, header=1, index_col=10, skiprows=1)
            data[0].drop(data[0].columns[[10, 11]], axis=1, inplace=True)#remove useless columns (all are empty)
            if writeToFile:
                data[0].to_excel(DATA_FOLDER_PATH + path) #save to xls file
            df = data[0]
    else:
        df = pd.read_excel(DATA_FOLDER_PATH + path, sheetname=0, header=0, skiprows=0, skip_footer=0, index_col=0)
    
    return df

We can now start using those functions to collect data. 
Example below:

In [188]:
test = query_epfl("Informatique", 2007,2008, "Bachelor semestre", 1, "d'automne")
test

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange
No Sciper,,,,,,,,,,
169569,Monsieur,Arévalo Christian,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
174905,Monsieur,Aubelle Flavien,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
173922,Monsieur,Badoud Morgan,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
179406,Monsieur,Baeriswyl Jonathan,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
179428,Monsieur,Barroco Michael,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
179324,Monsieur,Belfis Nicolas,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
174597,Monsieur,Beliaev Stanislav,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
179449,Monsieur,Bindschaedler Vincent,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN
178553,Monsieur,Bloch Marc-Olivier,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN


We're now going to collect and merge every single semester of every single year. We'll append columns "year", "semester".

Together with a student's sciper, we'll use this to make unique indexes. We could've used the name but it's possible that two students with the same name would be in the same class in the same year and semestre. 

In [192]:
frames = []
for year in range(2007, 2017):
    for semester in range (1, 10):
        if(semester % 2 == 0 ):
            season="de printemps"
        else:
            season="d'automne"
            
        df = query_epfl("Informatique", year,year+1, "Bachelor semestre", semester, season)
        #df.columns = ["civility", "name", "status", "sciper"]
        #df['year'] = year
        #df['semester'] = semester
        #df.set_index(['sciper', 'year', 'semester'])
        frames.append(df)
result = pd.concat(frames)

We make sure the index is unique

In [193]:
print(result.index.is_unique)

False


In [194]:
display(result)

,Civilité,Ecole Echange,Filière opt.,"Informatique, 2009-2010, Admission printemps (1 ét.)","Informatique, 2012-2013, Admission printemps (4 ét.)",Mineur,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,...,Type Echange,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
No Sciper,,,,,,,,,,,,,,,,,,,,,
169569,Monsieur,NaN,NaN,NaN,NaN,NaN,Arévalo Christian,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174905,Monsieur,NaN,NaN,NaN,NaN,NaN,Aubelle Flavien,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173922,Monsieur,NaN,NaN,NaN,NaN,NaN,Badoud Morgan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179406,Monsieur,NaN,NaN,NaN,NaN,NaN,Baeriswyl Jonathan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179428,Monsieur,NaN,NaN,NaN,NaN,NaN,Barroco Michael,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179324,Monsieur,NaN,NaN,NaN,NaN,NaN,Belfis Nicolas,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174597,Monsieur,NaN,NaN,NaN,NaN,NaN,Beliaev Stanislav,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179449,Monsieur,NaN,NaN,NaN,NaN,NaN,Bindschaedler Vincent,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178553,Monsieur,NaN,NaN,NaN,NaN,NaN,Bloch Marc-Olivier,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
